In [42]:
import pandas as pd
import csv
import json

In [43]:
stategeo_df = pd.read_csv("Raw Data/state_geojson.csv")
stategeo_df.head()


,type,properties,geometry
0,Feature,"{'GEO_ID': '0400000US23', 'STATE': '23', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-6..."
1,Feature,"{'GEO_ID': '0400000US25', 'STATE': '25', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
2,Feature,"{'GEO_ID': '0400000US26', 'STATE': '26', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-8..."
3,Feature,"{'GEO_ID': '0400000US30', 'STATE': '30', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-104.057..."
4,Feature,"{'GEO_ID': '0400000US32', 'STATE': '32', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-114.050..."


In [44]:
state = ['Maine', 'Massachusetts', 'Michigan', 'Montana', 'Nevada', 'New Jersey', 'New York', 'North Carolina', 'Ohio', 'Pennsylvania', 'Rhode Island', 'Tennessee', 'Texas', 'Utah', 
'Washington', 'Wisconsin', 'Puerto Rico', 'Maryland', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida',
'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Lousiana', 'Minnesota', 'Mississippi', 'Missouri', 'Nebraska', 'New Hampshire', 
'New Mexico', 'North Dakota', 'Oklahoma', 'Oregon', 'South Carolina', 'South Dakota', 'Vermont', 'Virginia', 'West Virginia', 'Wyoming']

In [50]:
stategeo_df['NAME'] = state
stategeo_df.head()

,type,properties,geometry,state,NAME
0,Feature,"{'GEO_ID': '0400000US23', 'STATE': '23', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-6...",Maine,Maine
1,Feature,"{'GEO_ID': '0400000US25', 'STATE': '25', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Massachusetts,Massachusetts
2,Feature,"{'GEO_ID': '0400000US26', 'STATE': '26', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-8...",Michigan,Michigan
3,Feature,"{'GEO_ID': '0400000US30', 'STATE': '30', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-104.057...",Montana,Montana
4,Feature,"{'GEO_ID': '0400000US32', 'STATE': '32', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-114.050...",Nevada,Nevada


In [46]:
state_df = pd.read_csv("Raw Data/state_data.csv", encoding="utf-8")
state_df.head()

,state,NAME,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,TotalPop,VotingAgeCitizen,Men %,...,Hispanic %,White %,Black %,Native %,Asian %,Pacific %,Biden %,Trump %,Republican,Democrat
0,AK,Alaska,260983,148624,102080,16705,83,738565,530553,0.523067,...,0.054854,0.521503,0.012701,0.340637,0.064436,0.005868,0.391137,0.569478,True,False
1,AL,Alabama,2309900,1434159,843473,193985,2973,4850771,3651914,0.484625,...,0.033495,0.661761,0.291164,0.006169,0.007229,0.000182,0.365156,0.620875,True,False
2,AR,Arkansas,1209372,757052,418051,113057,1958,2977944,2183895,0.490826,...,0.051857,0.770472,0.164333,0.005501,0.007049,0.000788,0.345676,0.625988,True,False
3,AZ,Arizona,3366867,1651812,1663447,247473,5979,6809946,4690177,0.497075,...,0.314592,0.517689,0.018469,0.134786,0.013173,0.001290,0.494064,0.490608,False,True
4,CA,California,16092730,5416035,10339137,935878,17671,38982847,24970109,0.496797,...,0.310681,0.565784,0.030425,0.017047,0.073052,0.003010,0.642473,0.336552,False,True


In [47]:
state_df['hispanic_pop'] = round(state_df['TotalPop']*state_df['Hispanic %'])

state_df['white_pop'] = round(state_df['TotalPop']*state_df['White %'])

state_df['black_pop'] = round(state_df['TotalPop']*state_df['Black %'])

state_df['native_pop'] = round(state_df['TotalPop']*state_df['Native %'])

state_df['asian_pop'] = round(state_df['TotalPop']*state_df['Asian %'])

state_df['pacific_pop'] = round(state_df['TotalPop']*state_df['Pacific %'])

In [54]:
merged_df = pd.merge(stategeo_df, state_df, on="NAME")
merged_df.head()
clean_state_df = merged_df.drop(['state_x'],axis=1)

clean_state_df["cases_per_100k"]= (clean_state_df["cases"]/clean_state_df["TotalPop"])*100000
clean_state_df["deaths_per_100k"]= (clean_state_df["deaths"]/clean_state_df["TotalPop"])*100000

clean_state_df.head()

,type,properties,geometry,NAME,state_y,total_votes20,votes20_Donald_Trump,votes20_Joe_Biden,cases,deaths,...,Republican,Democrat,hispanic_pop,white_pop,black_pop,native_pop,asian_pop,pacific_pop,cases_per_100k,deaths_per_100k
0,Feature,"{'GEO_ID': '0400000US23', 'STATE': '23', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-6...",Maine,ME,786337,341949,420357,6711,147,...,False,True,19145.0,1279415.0,10504.0,10420.0,10504.0,169.0,504.526530,11.051319
1,Feature,"{'GEO_ID': '0400000US25', 'STATE': '25', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-7...",Massachusetts,MA,3482216,1131042,2271763,159707,10013,...,False,True,630365.0,5455488.0,399548.0,20027.0,282388.0,1502.0,2352.327236,147.481655
2,Feature,"{'GEO_ID': '0400000US26', 'STATE': '26', 'NAME...","{'type': 'MultiPolygon', 'coordinates': [[[[-8...",Michigan,MI,5531329,2647004,2795649,197406,7699,...,False,True,351495.0,8949166.0,393596.0,128261.0,101336.0,1713.0,1988.863509,77.567349
3,Feature,"{'GEO_ID': '0400000US30', 'STATE': '30', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-104.057...",Montana,MT,602781,341765,243716,33495,376,...,True,False,30486.0,910095.0,2328.0,81715.0,4918.0,319.0,3252.377503,36.509746
4,Feature,"{'GEO_ID': '0400000US32', 'STATE': '32', 'NAME...","{'type': 'Polygon', 'coordinates': [[[-114.050...",Nevada,NV,1337217,635017,671887,101477,1781,...,False,True,510752.0,2132180.0,64538.0,113809.0,59854.0,6593.0,3514.081154,61.674848


In [55]:
clean_state_df.to_csv('CleanData/state_data.csv')